In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from umap import UMAP
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

from scipy.stats import zscore
from scipy.io import loadmat, savemat
import mat73
import os
import time

# conda activate shinlab # installed umap here
# DON'T FORGET: to use synology, need to run the following line in terminal
# net use S: \\shinlab\ShinLab
print('DON''T FORGET: to use synology, need to run in terminal: \"net use S: \\shinlab\ShinLab\" ')


DONT FORGET: to use synology, need to run in terminal: "net use S: \shinlab\ShinLab" 


In [2]:
# todo: add 10-fold cross validation and preprocessing (mean-centering/z-scoring) based on train set

def semi_supervised_umap_knn(data, labels, probedata, n_neighbors=20, n_components=2, n_splits=2, opt_supervise_umap=True, random_state=42):
    """
    Perform semi-supervised UMAP with kNN decoding, incorporating negative sampling and advanced validation.

    Parameters:
        data (numpy.ndarray): High-dimensional input data. NtrialsXNneurons
        labels (numpy.ndarray): Labels for supervised UMAP training. NtrialsX1
        n_neighbors (int): Number of neighbors for UMAP and kNN.
        n_components (int): Number of UMAP dimensions.
        test_size (float): Fraction of data for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        dict: Results including accuracy, confusion matrix, and embeddings.
    """

    opt_meancenter_embeddings = False    
    
    # todo: preprocess (mean-centering/z-scoring) based on train set
    X_probe = probedata
    
    # # Split data into training and test sets
    # X_train, X_test, y_train, y_test = train_test_split(
    #     data, labels, test_size=1.0/n_splits, random_state=random_state)

    # Create StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    train_index_list = []
    test_index_list = []
    y_test_list = []    
    accuracies = []
    confusion_matrices = []
    
    train_embeddings_list = []
    test_embeddings_list = []
    probe_embeddings_list = []
    
    y_train_pred_list = []
    y_test_pred_list = []
    y_probe_pred_list = []
    
    for train_index, test_index in skf.split(data, labels):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
            
        # Apply supervised UMAP with adjusted hyperparameters
        umap_model = UMAP(
            n_neighbors=n_neighbors,
            n_components=n_components,
            metric="cosine", # note, this makes mean-centering unnecessary
            output_metric="euclidean",
            learning_rate=1.0,
            init="spectral",
            min_dist=0.1,
            spread=1.0,
            repulsion_strength=1.0,
            negative_sample_rate=5,
            target_metric="categorical",
            dens_lambda=2.0,
            dens_frac=0.3,
            dens_var_shift=0.1,
            random_state=random_state
        )
        
        if opt_supervise_umap:
            # make umap supervised
            embeddings = umap_model.fit_transform(X_train, y=y_train)
        else:
            embeddings = umap_model.fit_transform(X_train)
    
        # to make umap unsupervised, use code below instead
        # embeddings = umap_model.fit_transform(X_train)
        
        if opt_meancenter_embeddings:
            train_embeddings = embeddings.copy()
            train_embeddings = train_embeddings - np.mean(embeddings, axis=0)
        else:
            train_embeddings = embeddings.copy()
            
        # Train kNN classifier on UMAP embeddings
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(train_embeddings, y_train)
    
        # Transform test data using the same UMAP model
        test_embeddings = umap_model.transform(X_test)
    
        if opt_meancenter_embeddings:
            test_embeddings = test_embeddings - np.mean(embeddings, axis=0)
        
        # Predict train/test labels
        y_train_pred = knn.predict(train_embeddings)
        y_test_pred = knn.predict(test_embeddings)
    
            
        # probe data
        probe_embeddings = umap_model.transform(X_probe)
        if opt_meancenter_embeddings:
            probe_embeddings = probe_embeddings - np.mean(embeddings, axis=0)
        y_probe_pred = knn.predict(probe_embeddings)
        
        # Compute accuracy and confusion matrix
        accuracy = accuracy_score(y_test, y_test_pred)
        conf_matrix = confusion_matrix(y_test, y_test_pred)

        
        train_index_list.append(train_index)
        test_index_list.append(test_index)    
        y_test_list.append(y_test)    
        accuracies.append(accuracy)
        confusion_matrices.append(conf_matrix)
        
        train_embeddings_list.append(train_embeddings)
        test_embeddings_list.append(test_embeddings)
        probe_embeddings_list.append(probe_embeddings)
                
        y_train_pred_list.append(y_train_pred)
        y_test_pred_list.append(y_test_pred)
        y_probe_pred_list.append(y_probe_pred)
        
    return {
        "train_index": train_index_list,
        "test_index": test_index_list,
        "test_truelabels": y_test_list,
        "accuracy": accuracies,
        "confusion_matrix": confusion_matrices,
        "train_embeddings": train_embeddings_list,
        "test_embeddings": test_embeddings_list,
        "probe_embeddings": probe_embeddings_list,
        "train_predlabels": y_train_pred_list,
        "test_predlabels": y_test_pred_list,
        "probe_predlabels": y_probe_pred_list,
    }


In [3]:
# todo: add 10-fold cross validation and preprocessing (mean-centering/z-scoring) based on train set

def semi_supervised_umap(data, labels, n_neighbors=20, n_components=2, opt_supervise_umap=True, random_state=42):
    """
    Perform semi-supervised UMAP with kNN decoding, incorporating negative sampling and advanced validation.

    Parameters:
        data (numpy.ndarray): High-dimensional input data. NtrialsXNneurons
        labels (numpy.ndarray): Labels for supervised UMAP training. NtrialsX1
        n_neighbors (int): Number of neighbors for UMAP and kNN.
        n_components (int): Number of UMAP dimensions.
        test_size (float): Fraction of data for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        dict: Results including accuracy, confusion matrix, and embeddings.
    """

    # Apply supervised UMAP with adjusted hyperparameters
    umap_model = UMAP(
        n_neighbors=n_neighbors,
        n_components=n_components,
        metric="cosine", # note, this makes mean-centering unnecessary
        output_metric="euclidean",
        learning_rate=1.0,
        init="spectral",
        min_dist=0.1,
        spread=1.0,
        repulsion_strength=1.0,
        negative_sample_rate=5,
        target_metric="categorical",
        dens_lambda=2.0,
        dens_frac=0.3,
        dens_var_shift=0.1,
        random_state=random_state
    )
    
    if opt_supervise_umap:
        # make umap supervised
        embeddings = umap_model.fit_transform(data, y=labels)
    else:
        embeddings = umap_model.fit_transform(data)
    
        
    return {
        "embeddings": embeddings
    }


In [5]:
# datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# # List all files and directories in the data directory
# nwbdir = os.listdir(datadir)

# nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
#     'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
#     'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

# ises = 0
# pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
# print(ises, pathpp)

# fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
# try:
#     spkcntlmlv = loadmat(fn2load)
# except:
#     spkcntlmlv = mat73.loadmat(fn2load)
    
# Nslopes = len(spkcntlmlv['disperses'][0])
# for islope in np.arange(10,Nslopes,2): #np.arange(Nslopes):
#     lmlvslope = spkcntlmlv['disperses'][0][islope]
#     print(lmlvslope)


0 \\shinlab\ShinLab\OpenScopeData\00248_v240130\\postprocessed\sub-619293\
0.0
0.20000000000000018
0.40000000000000013
0.5999999999999999
0.7999999999999998
1.0
1.2
1.4
1.6
1.8
2.0


In [6]:
# Define the data directory
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

for ises in np.arange(len(nwbsessions)):
    pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
    print(ises, pathpp)
    
    fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
    try:
        spkcntlmlv = loadmat(fn2load)
    except:
        spkcntlmlv = mat73.loadmat(fn2load)
    
    
    blocktrialorder = spkcntlmlv['trialorder'].flatten()
    vislabel = blocktrialorder
    
    Rblock = spkcntlmlv['spkcntorig']
    Rdata = Rblock.copy()
    
    tic = time.time()
    UMAPorig = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_splits=2, opt_supervise_umap=True)
    UMAPorig_unsup = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_splits=2, opt_supervise_umap=False)
    UMAPorigall = semi_supervised_umap(Rdata, vislabel, opt_supervise_umap=True)
    UMAPorigall_unsup = semi_supervised_umap(Rdata, vislabel, opt_supervise_umap=False)
    toc = time.time() - tic
    print('Elapsed time: ', toc)

    lmlvslope_list = []
    UMAPlmlv_list = []
    UMAPlmlv_unsup_list = []
    UMAPlmlvall_list = []
    UMAPlmlvall_unsup_list = []
    Nslopes = len(spkcntlmlv['disperses'][0])
    for islope in np.arange(10,Nslopes,2): #np.arange(Nslopes):
        lmlvslope = spkcntlmlv['disperses'][0][islope]
        print(lmlvslope)
        Rblock = spkcntlmlv['spkcntlmlv'][0][islope]
        Rblock[np.isnan(Rblock)] = 0
        Rdata = Rblock.copy()
    
        tic = time.time()
        UMAPslope = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_splits=2, opt_supervise_umap=True)
        UMAPslope_unsup = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_splits=2, opt_supervise_umap=False)
        
        UMAPallslope = semi_supervised_umap(Rdata, vislabel, opt_supervise_umap=True)
        UMAPallslope_unsup = semi_supervised_umap(Rdata, vislabel, opt_supervise_umap=False)
        toc = time.time() - tic
        print('Elapsed time: ', toc)

        lmlvslope_list.append(lmlvslope)
        UMAPlmlv_list.append(UMAPslope)
        UMAPlmlv_unsup_list.append(UMAPslope_unsup)
        UMAPlmlvall_list.append(UMAPallslope)
        UMAPlmlvall_unsup_list.append(UMAPallslope_unsup)
    
    UMAP_dict = {
        'lmlvslope_list': lmlvslope_list,
        'UMAPorig': UMAPorig,
        'UMAPorig_unsup': UMAPorig_unsup,
        'UMAPorigall': UMAPorigall,
        'UMAPorigall_unsup': UMAPorigall_unsup,
        
        'UMAPlmlv': UMAPlmlv_list,
        'UMAPlmlv_unsup': UMAPlmlv_unsup_list,
        'UMAPlmlvall': UMAPlmlvall_list,
        'UMAPlmlvall_unsup': UMAPlmlvall_unsup_list,
    }
    
    fn2save = pathpp + 'UMAP_V1RS_lmlvslopes.mat'
    savemat(fn2save, UMAP_dict)

0 \\shinlab\ShinLab\OpenScopeData\00248_v240130\\postprocessed\sub-619293\


C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed

Elapsed time:  146.7312092781067
0.0


C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed

Elapsed time:  132.52939176559448
0.20000000000000018


...

Elapsed time:  137.82000374794006
1.8


C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed

Elapsed time:  137.3738887310028
2.0


C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\shinlab\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed

Elapsed time:  136.38056182861328
